In [17]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import fnmatch
import string
from urllib.parse import urlparse
import contractions

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('words')
from nltk.tokenize import sent_tokenize, word_tokenize, TweetTokenizer
from nltk.corpus import words, stopwords
from nltk.metrics.distance import jaccard_distance, edit_distance
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/arjunkhanchandani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/arjunkhanchandani/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/arjunkhanchandani/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [44]:
tweets_df = pd.read_csv('/Users/arjunkhanchandani/Desktop/twitter_data_analysis-main/data/20221109_054000_tweets.csv')
tweets_df.head()

user_id  tweet_id         username                location  following  \
0        1         1  GopinathNagend1        Hyderabad, India       1080   
1        2         2       pspatilsbi              Bangalore         325   
2        3         3         ththegde  Kandivali East, Mumbai        582   
3        4         4  ahmed_chowdhary    Andheri West, Mumbai         30   
4        5         5      rupz_boruah           Chabua, India         14   

   followers             twt_created_at  total_tweets  retweet_count  \
0        304  2022-11-08 22:28:05+00:00         61966              0   
1         25  2022-11-08 22:08:44+00:00          2704              0   
2         57  2022-11-08 22:00:49+00:00          1969              0   
3         21  2022-11-08 21:59:32+00:00           332              0   
4         33  2022-11-08 21:54:37+00:00           309              0   

                                                text  \
0  @isvelan @PrapancaMAnudan Why not ? When @Arvi...   
1  This was, this is and....this will be the agen...   
2  @PMOIndia @nsitharaman please allow citizens t...   
3  @ANI Downfall of #IndianEconomy I felt In #Mumbai   
4  Please take necessary action against neurologi...   

                                            hashtags  \
0                                                 []   
1                                                 []   
2                                                 []   
3  [{'text': 'IndianEconomy', 'indices': [17, 31]...   
4  [{'text': 'Dr_Dhrubajyoti_Kurmi', 'indices': [...   

                                            mentions  
0  [{'screen_name': 'isvelan', 'name': 'üáÆüá≥ ...  
1  [{'screen_name': 'INCIndia', 'name': 'Congress...  
2  [{'screen_name': 'PMOIndia', 'name': 'PMO Indi...  
3  [{'screen_name': 'ANI', 'name': 'ANI', 'id': 3...  
4  [{'screen_name': 'MoHFW_INDIA', 'name': 'Minis...

In [45]:
tweets_df.shape

(2811, 12)

In [50]:
print('Shape of dataset before removal of duplicates is {}'.format(tweets_df.shape))
tweets_df.drop_duplicates(subset=['text'], inplace=True)
print('Shape of dataset after removal of duplicates is {}'.format(tweets_df.shape))

Shape of dataset before removal of duplicates is (961, 12)
Shape of dataset after removal of duplicates is (961, 12)


In [51]:
tweets_df.dtypes

user_id            int64
tweet_id           int64
username          object
location          object
following          int64
followers          int64
twt_created_at    object
total_tweets       int64
retweet_count      int64
text              object
hashtags          object
mentions          object
dtype: object

In [52]:
def data_preprocessing(df):
    """
    One function to rule them all, 
    one function to find them, 
    One function to bring them all, 
    and in the darkness bind them; 
    """
    print('Shape of dataset before removal of tweets with less than 5 words is {}'.format(df.shape))
    
    for tweets in df.loc[:,'text']:
        # count+=1
        # print(tweets)
        tokenizer = TweetTokenizer()
        tweet_id = df.loc[df['text'] == tweets, 'tweet_id'].values[0]
        # print(tweet_id)
        
        # removing links
        list_words = tokenizer.tokenize(tweets)
        x = [word for word in list_words if not urlparse(word).scheme]
        tweets = ' '.join(x)

        # contractions handling
        list_words = tokenizer.tokenize(tweets)
        new_list_words = []
        for word in list_words:
            new_list_words.append(contractions.fix(word))
        list_words = new_list_words
        del(new_list_words)
        tweets = ' '.join(list_words)
        
        # adding space between words and punctuations
        tweets = tweets.replace(',', ' ,').replace('.', ' .').replace('?', ' ?').replace('!', ' !')
        
        # removing hashtags and mentions
        list_words = tokenizer.tokenize(tweets)
        list_words = [word for word in list_words if word[0] not in ('#', '@')]
        list_words = [word for word in list_words if word[0] not in ('▪')]
        tweets = ' '.join(list_words)
        
        # removing punctuations
        tweets = tweets.translate(str.maketrans('', '', string.punctuation))
        
        #removing emojis
        tweets = re.sub(r'[^\x00-\x7F]+', ' ', tweets)
        
        #lower case
        tweets = tweets.lower()
        
        #remove numbers
        tweets = re.sub(r'\d+', '', tweets)
        tweets = re.sub(' +', ' ', tweets)
        
        #removing stopwords
        list_words = tokenizer.tokenize(tweets)
        filtered_words = [word for word in list_words if word not in stopwords.words('english')]
        tweets = ' '.join(filtered_words)
        del(filtered_words)
        
        #lemmatization
        lem = WordNetLemmatizer()
        list_words = tokenizer.tokenize(tweets)
        for word in list_words:
            list_words = list(map(lambda x: x.replace(word, lem.lemmatize(word)), list_words))
        tweets = ' '.join(list_words)
        
        #removing individual letters
        list_words = tokenizer.tokenize(tweets)
        filtered_words = [word for word in list_words if len(word)>2]
        tweets = ' '.join(filtered_words)
        del(filtered_words)
        
        # updating tweets in dataframe
        df.loc[df['tweet_id']==tweet_id, 'text'] = tweets
        
        #remove small tweets
        list_words = tokenizer.tokenize(tweets)
        if len(list_words) <= 5:
            ind_num = df[df['tweet_id']==tweet_id].index
            df.drop(ind_num, inplace=True)
        # break
    print('Shape of dataset after removal of tweets with less than 5 words is {}'.format(df.shape))
    
    return df
        
        
        

In [53]:
tweets_df = data_preprocessing(tweets_df)

Shape of dataset before removal of tweets with less than 5 words is (961, 12)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [54]:
# converting tweets_df into a csv file
filename = '/Users/arjunkhanchandani/Desktop/twitter_data_analysis-main/data/final_dataset.csv'
tweets_df.to_csv(filename, index=False)